In [ ]:
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from skimage.io import imread
from shutil import copyfile
#import tensorflow_probability as tfp
print(tf.__version__)
import matplotlib.pyplot as plt
plt.gray()
from sklearn.metrics import accuracy_score
#import tensorflow_probability as tfp
import random
import json
import datetime


BATCH_SIZE = 32.
EPOCHS = 512.

batch_size = BATCH_SIZE
epochs = EPOCHS
learning_rate=0.001
CHANNELS=1
NLAYERS=1.
NFILTERS=48.
KSIZE=13
SUBSPACE=12
PATIENCE_ES=40
PATIENCE_RP=5


class NeymanScott:
    """
    Neyman-Scott point process using a Poisson variable for the number of parent points, uniform for
    the number of daughter points and Pareto distribution for the distance from the daughter points to
    the parent.
    """
    def __init__(self,
                 poisson_mean: float,
                 daughter_max: int,
                 pareto_alpha: float,
                 pareto_scale: float,
                 size: (int, int)):
        """
        :param poisson_mean: mean of the number of parent points
        :param daughter_max: maximum number of daughters per parent points
        :param pareto_alpha: alpha parameter of the Pareto distribution
        :param pareto_scale: scale used in the Pareto distribution. This parameter is
            applied before resizing the points from the [0, 1] interval to the size of the image.
        :param size: rescale the output to this size
        """
        self.poisson_mean = poisson_mean
        self.daughter_max = daughter_max
        self.pareto_alpha = pareto_alpha
        self.pareto_scale = pareto_scale
        self.size = np.array([size])
        self.generator = np.random.Generator(np.random.PCG64())

    def __call__(self):
        num_parents = self.generator.poisson(lam=self.poisson_mean)
        parents = self.generator.random((num_parents, 2))
        num_daughters = self.generator.integers(1, self.daughter_max, num_parents)
        points = np.empty((0, 2))

        for i in range(num_parents):
            # normalizes the pareto II distribution
            dist = self.generator.pareto(self.pareto_alpha, (num_daughters[i], 1))
            dist = (dist + 1) * self.pareto_scale
            angle = self.generator.uniform(0., 2 * np.pi, (num_daughters[i],))
            positions = np.stack([np.cos(angle), np.sin(angle)], 1)
            positions *= dist
            positions += parents[i, np.newaxis, :]
            points = np.concatenate([points, positions])
        # remove points outside the set [0, 1] x [0, 1]
        valid_points = np.logical_and(
            np.logical_and(0. <= points[:, 0], points[:, 0] <= 1.),
            np.logical_and(0. <= points[:, 1], points[:, 1] <= 1.)
        )
        points = points[valid_points, :]
        # scale to the image size
        points = points * self.size
        return points



NSAMPLES_TRAINING=2024*2
IMG_SIZE=128
poisson_mean=100
daughter_max=50
pareto_scale=.02
pareto_alpha=1. #GENERATION ON IT
gen = NeymanScott(poisson_mean, daughter_max, pareto_alpha, pareto_scale, (IMG_SIZE, IMG_SIZE))

generatedata
if generatedata==1:
    listIm=[]
    listY=[]
    for i in range(NSAMPLES_TRAINING):
        pareto_alpha=gen.generator.random(1)*10
        gen = NeymanScott(poisson_mean, daughter_max, pareto_alpha, pareto_scale, (IMG_SIZE, IMG_SIZE))
        points = gen()
        I=np.zeros([IMG_SIZE,IMG_SIZE])
        I[np.int64(np.floor(points[:, 0])), np.int64(np.floor(points[:, 1]))]=1
        listIm.append(I)
        listY.append(pareto_alpha)
    listIm=np.stack(listIm)
    listY=np.stack(listY)

    NSAMPLES_VALIDATION=512
    listImVal=[]
    listYVal=[]
    for i in range(NSAMPLES_VALIDATION):
        pareto_alpha=gen.generator.random(1)*10
        gen = NeymanScott(poisson_mean, daughter_max, pareto_alpha, pareto_scale, (IMG_SIZE, IMG_SIZE))
        points = gen()
        I=np.zeros([IMG_SIZE,IMG_SIZE])
        I[np.int64(np.floor(points[:, 0])), np.int64(np.floor(points[:, 1]))]=1
        listImVal.append(I)
        listYVal.append(pareto_alpha)

    listImVal=np.stack(listImVal)
    listYVal=np.stack(listYVal)
    np.save(args.pathoutput+'listIm.npy',listIm)
    np.save(args.pathoutput+'listImVal.npy',listImVal)
    np.save(args.pathoutput+'listY.npy',listY)
    np.save(args.pathoutput+'listYVal.npy',listYVal)
else:
    listIm=np.load(args.pathoutput+'listIm.npy')
    listImVal=np.load(args.pathoutput+'listImVal.npy')
    listY=np.load(args.pathoutput+'listY.npy')
    listYVal=np.load(args.pathoutput+'listYVal.npy')

print('listIm.shape',listIm.shape)
print('listImVal.shape',listImVal.shape)
print('listY.shape',listY.shape)
print('listYVal.shape',listYVal.shape)

listY=listY/9
listYVal=listYVal/9
